In [1]:
import numpy as np
import sympy as sp
import qiskit
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
import itertools as it

import circuit
from scipy.special import rel_entr

In [2]:
def prob_KL(num_qubits, num_gate):
    qc = circuit.Circuit(num_qubits)

    gate_list = ['h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'swap', 'cx', 'cz']
    for i in range(num_gate):
        rnd_gate = np.random.choice(gate_list)
        if rnd_gate in ['h', 'x', 'y', 'z']:
            wire = np.random.randint(num_qubits)
            getattr(qc, rnd_gate)(wire)
        elif rnd_gate in ['rx', 'ry', 'rz']:
            symbol_name = f"t_{i}"
            wire = np.random.randint(num_qubits)
            getattr(qc, rnd_gate)(symbol_name, wire)
        elif rnd_gate in ['swap', 'cx', 'cz']:
            wire1, wire2 = np.random.choice(range(num_qubits), size=2, replace=False)
            getattr(qc, rnd_gate)(wire1, wire2)

    symbol_dict = {}
    qiskit_dict = {}
    for i in range(len(qc.qiskit_circuit.parameters)):
        symbol_name = qc.qiskit_circuit.parameters[i].name
        rnd_radian  = 2 * np.pi * np.random.rand()
        # see https://stackoverflow.com/questions/73170057/sympy-evalf-for-real-symbol
        symbol_dict[sp.Symbol(symbol_name, real=True)] = rnd_radian
        qiskit_dict[qc.qiskit_circuit.parameters[i]] = rnd_radian
    q_circuit = qc.qiskit_circuit.bind_parameters(qiskit_dict)

    sv = Statevector.from_label(num_qubits * '0')
    sv = sv.evolve(q_circuit)
    sv = sv.reverse_qargs()
    qc.evolve_state()
    psi = np.array(sp.Abs(qc.final_state.evalf(subs=symbol_dict)), dtype=np.float).reshape(-1)
    qiskit_prob = np.around(np.array(sv.probabilities(), dtype=np.float), decimals=5)
    sympy_prob  = np.around(psi**2, decimals=5)
    return qiskit_prob, sympy_prob

In [3]:
for _ in range(10):
    qiskit_prob, sympy_prob = prob_KL(num_qubits=3, num_gate=10)
    # print(qiskit_prob)
    # print(sympy_prob)
    result_KL = np.sum(rel_entr(qiskit_prob, sympy_prob))
    if result_KL != 0:
        print(result_KL)